<a href="https://colab.research.google.com/github/johndaguio/CPEN65-1/blob/main/Lab_Exercise_2_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), param_grid=[{'n_neighbors': [3, 4, 5],'weights': ['uniform', 'distance']}],verbose=3)
grid_search.best_params_
grid_search.best_score_
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

NameError: ignored

In [52]:
from scipy.ndimage import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])
    image = X_train[1000]
    shifted_image_down = shift_image(image, 0, 5)
    shifted_image_left = shift_image(image, -5, 0)

    plt.figure(figsize=(12,3))
    plt.subplot(131)
    plt.title("Original", fontsize=14)
    plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")
    plt.subplot(132)
    plt.title("Shifted down", fontsize=14)
    plt.imshow(shifted_image_down.reshape(28, 28), interpolation="nearest", cmap="Greys")
    plt.subplot(133)
    plt.title("Shifted left", fontsize=14)
    plt.imshow(shifted_image_left.reshape(28, 28), interpolation="nearest", cmap="Greys")
    plt.show()

In [55]:
import os
import urllib.request

TITANIC_PATH = os.path.join("datasets", "titanic")
DOWNLOAD_URL = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/titanic/"

def fetch_titanic_data(url=DOWNLOAD_URL, path=TITANIC_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    for filename in ("train.csv", "test.csv"):
        filepath = os.path.join(path, filename)
        if not os.path.isfile(filepath):
            print("Downloading", filename)
            urllib.request.urlretrieve(url + filename, filepath)

fetch_titanic_data()    

import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

    train_data = load_titanic_data("train.csv")
    test_data = load_titanic_data("test.csv")
    train_data.head()
    train_data = train_data.set_index("PassengerId")
    test_data = test_data.set_index("PassengerId")
    train_data.info()
    train_data[train_data["Sex"]=="female"]["Age"].median()
    train_data.describe()
    train_data["Survived"].value_counts()
    train_data["Pclass"].value_counts()
    train_data["Sex"].value_counts()
    train_data["Embarked"].value_counts()
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler

    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())])
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer

    num_attribs = ["Age", "SibSp", "Parch", "Fare"]
    cat_attribs = ["Pclass", "Sex", "Embarked"]

    preprocess_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs),("cat", cat_pipeline, cat_attribs),])
    X_train = preprocess_pipeline.fit_transform(train_data[num_attribs + cat_attribs])
    y_train = train_data["Survived"]
    from sklearn.ensemble import RandomForestClassifier

    forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
    forest_clf.fit(X_train, y_train)

    X_test = preprocess_pipeline.transform(test_data[num_attribs + cat_attribs])
    y_pred = forest_clf.predict(X_test)

    from sklearn.model_selection import cross_val_score

    forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
    forest_scores.mean()

    from sklearn.svm import SVC

    svm_clf = SVC(gamma="auto")
    svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
    svm_scores.mean()

    import matplotlib.pyplot as plt

    plt.figure(figsize=(8, 4))
    plt.plot([1]*10, svm_scores, ".")
    plt.plot([2]*10, forest_scores, ".")
    plt.boxplot([svm_scores, forest_scores], labels=("SVM","Random Forest"))
    plt.ylabel("Accuracy", fontsize=14)
    plt.show()

    train_data["AgeBucket"] = train_data["Age"] // 15 * 15
    train_data[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

    train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
    train_data[["RelativesOnboard", "Survived"]].groupby(['RelativesOnboard']).mean()


